In [51]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
import gc
import catboost

In [25]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *
from attributes_recs import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df))
orgs_df.head()

16967 1252801


,org_id,city,average_bill,rating,rubrics_id,features_id,combined_id
0,16848414477362211020,spb,1000.0,4.479702,"[30776, 31375]","[1018, 1509, 11177, 11617, 11629, 11704, 11867...","[30776, 31375, 1018, 1509, 11177, 11617, 11629..."
1,1430604733320164116,spb,1000.0,4.514509,"[30776, 30770]","[246, 1018, 11617, 11629, 11704, 11867, 20422,...","[30776, 30770, 246, 1018, 11617, 11629, 11704,..."
2,9880309324224147401,spb,1000.0,3.884615,"[30770, 30774]","[1018, 11177, 11617, 11629, 11704, 11867, 2042...","[30770, 30774, 1018, 11177, 11617, 11629, 1170..."
3,5617879987171966456,spb,1000.0,0.000000,"[30774, 30775]","[1018, 1509, 10596, 11177, 11629, 11634, 11704...","[30774, 30775, 1018, 1509, 10596, 11177, 11629..."
4,5241461680470612149,spb,1000.0,4.532468,[30776],"[1018, 11177, 11617, 11629, 11704, 11867, 2042...","[30776, 1018, 11177, 11617, 11629, 11704, 1186..."


In [6]:
reviews = load_reviews("reviews.csv",users_df=users_df, orgs_df=orgs_df)
print(f"{len(reviews)}")
reviews.head()

3640835


,user_id,org_id,rating,ts,aspects,good,user_city,in_test,org_city,travel
0,16998268288908323644,7184895086928047809,2.0,105,[],0,msk,0,msk,0
1,3121447338909258868,7184895086928047809,5.0,464,[],1,msk,0,msk,0
2,1970649778250883025,7184895086928047809,3.0,789,[],0,msk,0,msk,0
3,7554889464530643866,7184895086928047809,4.0,936,[],1,msk,0,msk,0
4,15907910894057053620,7184895086928047809,1.0,1143,[],0,msk,0,msk,0


In [9]:
train_reviews, test_reviews = train_test_split(reviews, users_df, 
                                               min_ts=500,
                                               frac=0.5)
print(len(train_reviews), len(test_reviews))

3602574 22562


In [87]:
pandarallel.initialize(progress_bar=False)
mr=50
l2=1e5
result, encs = prepare_reviews_i2i(train_reviews, orgs_df,
                                   min_reviews_per_user=2,
                                   min_org_reviews=mr,
                                   min_travels_reviews=2,
                                   min_org_score=4.0)
print(len(result))
m = reviews_matrix(result, encs)
cc_mat = m.T * m
ease_sim = ease_solution(cc_mat, l2=l2).T
ease_preds = i2i_predict(ease_sim, test_reviews, encs, N=20)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
1275439


In [15]:
reviews_orgs = reviews.merge(orgs_df.drop(columns='city'), on='org_id',suffixes=("","_org"))

In [32]:
test_reviews.head()

,user_id,org_id,target,city
0,10000013865364765196,[3645433246359964078],"[11580662118654056420, 11580662118654056420]",msk
1,10000628293057756300,[9900109414165004332],[5002407858008059043],msk
2,10000647610132274861,"[9914151637425089008, 14650669308292398491]",[17142307353874899125],spb
3,10001317088848889093,[1171073807113510696],[8731609900869263597],msk
4,10001518595021913425,"[12678716723210274223, 14923852655393135787, 1...","[12046097390037935713, 5002407858008059043]",msk


In [28]:
from learn_to_rank import *

In [30]:
pool_builder = PoolBuilder(orgs_df, reviews)

In [ ]:
result, groups, label = pool_builder.build_pool(ease_preds, test_reviews)
result.head()

In [79]:
train_pool = Pool(data=result,group_id=groups,label=label)

In [83]:
def cb_params():
    return {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'Precision:top=10'],
    'loss_function':"YetiRank"
}

In [84]:
model = CatBoost(cb_params())
model.fit(train_pool)

CatBoostError: catboost/libs/metrics/metric.cpp:6456: All train targets are equal